In [1]:
# --- Cell 1: Install Ultralytics (YOLOv8) ---
# Run this cell once to install the necessary library.
!pip install ultralytics
print(">>> Ultralytics installed successfully.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.4 MB/s eta 0:00:00
>>> Ultralytics installed successfully.


In [2]:
# --- Cell 2: Unzip Your Dataset & Fix Paths ---
import os
import shutil
from pathlib import Path

print(">>> [Step 2/6] Unzipping dataset...")
# !!! Make sure this name matches your uploaded ZIP file !!!
zip_file_name = "Dental Caries X-ray.v6199i.yolov8-obb.zip"
unzip_dir = Path("/content/dataset")
sub_dir_name = "Dental Caries X-ray.v6199i.yolov8-obb" # The folder *inside* the zip
sub_dir = unzip_dir / sub_dir_name

# Unzip the file
!unzip -q "{zip_file_name}" -d "{unzip_dir}"

# Move files from sub-directory up to the main /dataset directory for simple paths
print(f">>> Moving files from {sub_dir} to {unzip_dir}...")
if sub_dir.exists() and sub_dir.is_dir():
    try:
        for item in sub_dir.iterdir():
            # Use str() to ensure compatibility with shutil.move
            shutil.move(str(item), str(unzip_dir / item.name))
        sub_dir.rmdir()
        print(">>> Dataset unzipped and paths fixed.")
    except Exception as e:
        print(f"Error moving files: {e}. Check if files were already moved.")
else:
     print(f">>> Subdirectory {sub_dir_name} not found, assuming files are already in {unzip_dir}.")

>>> [Step 2/6] Unzipping dataset...
>>> Moving files from /content/dataset/Dental Caries X-ray.v6199i.yolov8-obb to /content/dataset...
>>> Dataset unzipped and paths fixed.


In [3]:
# --- Cell 3: (NEW) Clean and MERGE Label Files ---
# This cell loops through all .txt files.
# It MERGES ALL classes (0, 1, 2, 3, 4, etc.) into a
# single class (ID '0') which we will call 'Caries'.

import os
from pathlib import Path

print(">>> [Step 3/6] Starting label MERGE process...")
NEW_CLASS_ID = '0'    # The new, single ID for ALL detections

label_dirs = [
    Path("/content/dataset/train/labels"),
    Path("/content/dataset/valid/labels"),
    Path("/content/dataset/test/labels")
]

file_count = 0
total_line_count = 0

for dir_path in label_dirs:
    print(f"--- Merging labels in {dir_path} ---")
    if not dir_path.exists():
        print(f"Warning: Directory not found {dir_path}, skipping.")
        continue

    for label_file in dir_path.glob("*.txt"):
        file_count += 1
        cleaned_lines = []
        try:
            with open(label_file, 'r') as f:
                lines = f.readlines()

            for line in lines:
                parts = line.strip().split()
                if not parts:
                    continue

                # Regardless of the original ID (0, 1, 3, 11, etc.)
                # change it to our NEW_CLASS_ID ('0')
                parts[0] = NEW_CLASS_ID
                cleaned_lines.append(" ".join(parts))
                total_line_count += 1

            # Overwrite the file with the merged lines
            with open(label_file, 'w') as f:
                f.write("\n".join(cleaned_lines))
        except Exception as e:
            print(f"Error processing file {label_file}: {e}")

print(f"--- Label Merging Finished! ---")
print(f"Processed {file_count} files.")
print(f"Found and MERGED {total_line_count} total labels into a single 'Caries' class.")

>>> [Step 3/6] Starting label MERGE process...
--- Merging labels in /content/dataset/train/labels ---
--- Merging labels in /content/dataset/valid/labels ---
--- Merging labels in /content/dataset/test/labels ---
--- Label Merging Finished! ---
Processed 1483 files.
Found and MERGED 2706 total labels into a single 'Caries' class.


In [4]:
# --- Cell 4: Create the Correct 1-Class data.yaml ---
# This cell overwrites the old data.yaml with a new,
# clean version that only defines our one class: 'Caries'.

from pathlib import Path

print("\n>>> [Step 4/6] Creating new 1-class data.yaml...")
yaml_path = Path("/content/dataset/data.yaml")

# This is the new, correct config for a 1-class model
yaml_content = """
# Paths are relative to this file's location
train: train/images
val: valid/images
test: test/images

# nc: Number of Classes (We merged everything into 1)
nc: 1

# The name of our one and only class (ID 0)
names:
  - Caries
"""

try:
    with open(yaml_path, 'w') as f:
        f.write(yaml_content)
    print(f">>> New data.yaml created at {yaml_path}")
except Exception as e:
    print(f"Error writing data.yaml: {e}")


>>> [Step 4/6] Creating new 1-class data.yaml...
>>> New data.yaml created at /content/dataset/data.yaml


In [5]:
# --- Cell 5: Train with Manual Hyperparameter Control ---
# This is the "manual" training step.
# We are manually setting the optimizer, learning rate, and augmentations
# to demonstrate control over the training process.

from ultralytics import YOLO

print("\n>>> [Step 5/6] Starting training...")

# 1. Load a pre-trained model (yolov8-nano)
model = YOLO('yolov8n.pt')

# 2. Set the path to our new 1-class data.yaml
data_yaml_path = '/content/dataset/data.yaml'
project_name = 'caries_detection'
run_name = 'run_manual_tuning_v1' # Specific name for this experiment

# 3. Start training with manually set hyperparameters
results = model.train(
    data=data_yaml_path,

    # --- Experiment Control ---
    project=project_name,
    name=run_name,

    # --- Training Parameters (Manual) ---
    epochs=120,               # Manually set to 120 epochs
    batch=32,                 # Manually set batch size (adjust if GPU runs out of memory)
    patience=30,              # Stop if mAP doesn't improve for 25 epochs

    # --- Optimizer Parameters (Manual) ---
    optimizer='AdamW',        # Manually choosing the optimizer
    lr0=0.002,                # Manually setting a custom learning rate
    weight_decay=0.0005,

    # --- Augmentation Control (Manual) ---
    degrees=15,               # Add random rotation (+/- 15 degrees)
    translate=0.1,            # Add random translation
    scale=0.6,                # Add random scaling
    flipud=0.5                # Add a 50% chance of vertical flip
)

print("--- Advanced Training Finished! ---")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

>>> [Step 5/6] Starting ADVANCED (manual control) training...
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=15, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=120, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, 

In [6]:
# --- Cell 6: Download the New Model ---
# This cell downloads the 'best.pt' file from the specific run folder we just created ('run_manual_tuning_v1').

from google.colab import files
from pathlib import Path

print("\n>>> [Step 6/6] Starting download of the new 'best.pt'...")

# IMPORTANT: This path MUST match the 'project' and 'name' from Cell 5
best_model_path_str = "/content/caries_detection/run_manual_tuning_v1/weights/best.pt"
best_model_path = Path(best_model_path_str)

if best_model_path.exists():
    print(f">>> Success! Best model found at: {best_model_path_str}")
    files.download(best_model_path_str)
    print(">>> Download initiated.")
else:
    print(f"XXX ERROR: Could not find model at {best_model_path_str}")
    print("Please check the 'project' and 'name' in Cell 5 and update the path here.")


>>> [Step 6/6] Starting download of the new 'best.pt'...
>>> Success! Best model found at: /content/caries_detection/run_manual_tuning_v1/weights/best.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

>>> Download initiated.
